# Running Marrmot M01 using the ewatercycle system

Make sure your ewatercycle conda environment is [activated](https://github.com/eWaterCycle/ewatercycle#install).

### 1. Copy an example case

On https://github.com/wknoben/MARRMoT/tree/master/BMI/Config there is a example config file.

In [1]:
from pathlib import Path
forcing_file = Path('./BMI_testcase_m01_BuffaloRiver_TN_USA.mat')

In [ ]:
from ewatercycle.parametersetdb.datafiles import SubversionCopier
copier = SubversionCopier('https://github.com/wknoben/MARRMoT/trunk/BMI/Config/BMI_testcase_m01_BuffaloRiver_TN_USA.mat')
copier.save(forcing_file)

In [ ]:
from ewatercycle.forcing import load_foreign

In [3]:
forcing = load_foreign('marrmot',
                        directory='.',
                        start_time='1989-01-01T00:00:00Z',
                        end_time='1992-12-31T00:00:00Z',
                        forcing_info={
                            'forcing_file': 'BMI_testcase_m01_BuffaloRiver_TN_USA.mat'
                        })
forcing

MarrmotForcing(start_time='1989-01-01T00:00:00Z', end_time='1992-12-31T00:00:00Z', directory='.', shape=None, forcing_file='BMI_testcase_m01_BuffaloRiver_TN_USA.mat')

### 2. Verify your configuration settings

If you are working on a curated environment like https://lab.ewatercycle.org/, the default configuration should cover 90% of all use cases. However, for the other 10% here is a brief explanation of the relevant configuration settings. 

The ewatercycle configuration file is typcially stored in `~/.config/.ewatercycle/ewatercycle.yaml` or `/etc/ewatercycle.yaml` and can be imported with `from ewatercycle import CFG`. Relevant settings for any Marrmot model are:


In [4]:
import yaml
from ewatercycle.config import CFG

In [5]:
with open('./ewatercycle.yaml', 'w') as f:
    yaml.dump({
        'container_engine': 'docker', # or singularity
        'output_dir': './', # the directory where the model work_dir will be created
    }, f)

In [6]:
CFG.load_from_file('./ewatercycle.yaml')

### 3. Setting uip the model

In [7]:
import ewatercycle.models

In [8]:
ewatercycle.models.MarrmotM01.available_versions

['2020.11']

In [9]:
model = ewatercycle.models.MarrmotM14(version='2020.11', forcing=forcing)
model

/home/sarah/GitHub/ewatercycle/ewatercycle/models/marrmot.py:408: UserWarning: The length of parameters in forcing /home/sarah/GitHub/ewatercycle/examples/BMI_testcase_m01_BuffaloRiver_TN_USA.mat does not match the length of M14 parameters that is seven.
/home/sarah/GitHub/ewatercycle/ewatercycle/models/marrmot.py:413: UserWarning: The length of initial stores in forcing /home/sarah/GitHub/ewatercycle/examples/BMI_testcase_m01_BuffaloRiver_TN_USA.mat does not match the length of M14 iniatial stores that is two.


In [10]:
model.parameters

[('maximum_soil_moisture_storage', 1000.0),
 ('threshold_flow_generation_evap_change', 0.5),
 ('leakage_saturated_zone_flow_coefficient', 0.5),
 ('zero_deficit_base_flow_speed', 100.0),
 ('baseflow_coefficient', 0.5),
 ('gamma_distribution_chi_parameter', 4.25),
 ('gamma_distribution_phi_parameter', 2.5),
 ('initial_upper_zone_storage', 900.0),
 ('initial_saturated_zone_storage', 900.0),
 ('solver',
  Solver(name='createOdeApprox_IE', resnorm_tolerance=array([0.1]), resnorm_maxiter=array([6.]))),
 ('start time', '1989-01-01T00:00:00Z'),
 ('end time', '1992-12-31T00:00:00Z'),
 ('forcing_file',
  '/home/sarah/GitHub/ewatercycle/examples/BMI_testcase_m01_BuffaloRiver_TN_USA.mat')]

In [11]:
cfg_file, cfg_dir = model.setup(
    maximum_soil_moisture_storage=1234.0,
    end_time = "1989-01-10T00:00:00Z",
)
cfg_file, cfg_dir

(PosixPath('/home/sarah/GitHub/ewatercycle/examples/marrmot_20210603_153226/marrmot-m14_config.mat'),
 PosixPath('/home/sarah/GitHub/ewatercycle/examples/marrmot_20210603_153226'))

In [12]:
model.parameters

[('maximum_soil_moisture_storage', 1234.0),
 ('threshold_flow_generation_evap_change', 0.5),
 ('leakage_saturated_zone_flow_coefficient', 0.5),
 ('zero_deficit_base_flow_speed', 100.0),
 ('baseflow_coefficient', 0.5),
 ('gamma_distribution_chi_parameter', 4.25),
 ('gamma_distribution_phi_parameter', 2.5),
 ('initial_upper_zone_storage', 900.0),
 ('initial_saturated_zone_storage', 900.0),
 ('solver',
  Solver(name='createOdeApprox_IE', resnorm_tolerance=array([0.1]), resnorm_maxiter=array([6.]))),
 ('start time', '1989-01-01T00:00:00Z'),
 ('end time', '1989-01-10T00:00:00Z'),
 ('forcing_file',
  '/home/sarah/GitHub/ewatercycle/examples/BMI_testcase_m01_BuffaloRiver_TN_USA.mat')]

In [13]:
model.initialize(str(cfg_file))

In [14]:
model.update()

In [15]:
model.output_var_names

('P',
 'T',
 'Ep',
 'S(t)',
 'par',
 'sol_resnorm_tolerance',
 'sol_resnorm_maxiter',
 'flux_out_Q',
 'flux_out_Ea',
 'wb')

In [16]:
model.get_value('flux_out_Q')

array([1.66866573e-08])

In [17]:
model.get_value_as_xarray('flux_out_Q')

<xarray.DataArray 'flux_out_Q' (latitude: 1, longitude: 1)>
array([[1.66866573e-08]])
Coordinates:
  * longitude  (longitude) float64 87.49
  * latitude   (latitude) float64 35.29
    time       object 1989-01-02 00:00:00
Attributes:
    units:    mm day

In [18]:
while (model.time < model.end_time):
    model.update()
    print(model.get_value('flux_out_Q'))

[4.70585679e-194]
[3.76140319e-09]
[6.00284767e-194]
[6.77623459e-194]
[2.12776534e-08]
[8.69652115e-194]
[6.07162355e-08]
[3.23321345e-08]


In [19]:
model.finalize()

In [20]:
del model.bmi